In [6]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [7]:
import datetime
import pathlib
import math

from dateutil.relativedelta import relativedelta

In [8]:
year = datetime.date.today().year
month = datetime.date.today().month
today = datetime.date.today()
pre_month = today - relativedelta(months=1)
pre_month = datetime.date.strftime(pre_month, '%m')

In [9]:
print(pre_month)

10


In [10]:
url_web = "https://docs.google.com/spreadsheets/d/13T1tGrVGauIAo8wvZTkZuYtiEks4yGjUH1Rvqalim-o/edit#gid=1490271272" #PC&SP（アプリ面以外）
url_app = "https://docs.google.com/spreadsheets/d/1Hm7PMqq0npGb0qDXwqCGivoTE7QTThh7ae18QVuspE4/edit#gid=1812122780" #アプリ面

In [11]:
#シート上のデータ消去
for i in range(1,26):
    gs = GSheet()
    gs.clear_sheets(url_web, [i]) #PC&SP（アプリ面以外）
    gs.clear_sheets(url_app, [i]) #アプリ面

INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/sheets/v4/rest
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/13T1tGrVGauIAo8wvZTkZuYtiEks4yGjUH1Rvqalim-o?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/13T1tGrVGauIAo8wvZTkZuYtiEks4yGjUH1Rvqalim-o/values/Google%21A1%3AAA5002:clear?alt=json
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1Hm7PMqq0npGb0qDXwqCGivoTE7QTThh7ae18QVuspE4?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1Hm7PMqq0npGb0qDXwqCGivoTE7QTThh7ae18QVuspE4/values/Google%21A1%3AAA5002:clear?

In [12]:
#pandasの表示列数を増やす
pd.set_option('display.max_columns', 50)

In [13]:
%%slack_notify @yusuke_akada

#f-string
query = (
    f"""
    with tmp as (
    select 
    concat_ws('-',year,month) as yymm
    ,child_ssp_id as child_ssp
    ,case 
        when user_agent_os_family in ('iOS','Android') and is_app = 1 then 'SP_app'
        when user_agent_os_family in ('iOS','Android') then 'SP_web'
        ELSE 'PC' END as device
    ,domain
    ,sum(req) as req
    ,sum(0) as res
    ,sum(0) as imp
    ,sum(0) as click
    ,sum(0) as ctv1
    ,sum(0) as ctv2
    ,sum(0) gross_sales
    from dm.domain_req req
    where
    year = '{year}' and month = "{pre_month}" 
    group by 1,2,3,4
 
    union all

    select
    concat_ws('-',year,month) as yymm
    ,child_ssp_id as child_ssp
    ,case 
        when user_agent_os_family in ('iOS','Android') and is_app = 1 then 'SP_app'
        when user_agent_os_family in ('iOS','Android') then 'SP_web'
        ELSE 'PC' END as device
    ,domain
    ,sum(0) as req
    ,sum(res) as res
    ,sum(0) as imp
    ,sum(0) as click
    ,sum(0) as ctv1
    ,sum(0) as ctv2
    ,sum(0) gross_sales
    from dm.domain_res res
    where 
    year = '{year}' and month = "{pre_month}"
    group by 1,2,3,4
    
    union all
    
    select
    concat_ws('-',year,month) as yymm
    ,child_ssp
    ,case 
        when user_agent_os_family in ('iOS','Android') and is_app = 1 then 'SP_app'
        when user_agent_os_family in ('iOS','Android') then 'SP_web'
        ELSE 'PC' END as device
    ,domain
    ,sum(0) as req
    ,sum(0) as res
    ,sum(imp) as imp
    ,sum(click) as click
    ,sum(ctv1) as ctv1
    ,sum(ctv2) as ctv2
    ,cast(sum(win_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 as int) gross_sales
    from dm.domain_ads ads 
    inner join agency_console.campaign cp 
        on ads.strategy_id = cp.real_strategy_id 
    inner join agency_console.campaign_agency_margin cam 
        on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
    inner join agency_console.campaign_smn_margin csm 
        on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
    where 
    year = '{year}' and month = "{pre_month}"
    group by 1,2,3,4
    )
    , imp as (
    select 
    yymm
    ,child_ssp
    ,device
    ,domain
    ,sum(req) as req
    ,sum(res) as res
    ,sum(imp) as imp
    ,sum(click) as click
    ,sum(case when click > 0 then gross_sales/click else 0 end) as cpc
    ,sum(case when click > 0 then click/imp else 0 end) as ctr
    ,sum(gross_sales) as gross_sales
    ,(sum(gross_sales)/sum(imp))*1000 as cpm
    ,sum(ctv1) as ctv1
    ,sum(ctv2) as ctv2
    ,sum(case when ctv1 > 0 and click > 0 then ctv1/click else 0 end) as cvr1
    ,sum(case when (ctv1 > 0 or ctv2 > 0) then gross_sales/(ctv1+ctv2) else 0 end) as cpa
    ,sum(case when (ctv1 > 0 or ctv2 > 0) and click > 0 then (ctv1+ctv2)/click else 0 end) as cvr2_ctv1_2
    from tmp
    group by 1,2,3,4
    )
    , rank as(
    select 
    *
    ,ROW_NUMBER() OVER(PARTITION BY child_ssp ORDER BY imp desc) as rank
    from imp
    where imp >= 100
    )
    select
    *
    from rank
    where rank <= 5000

    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df = df.astype({'cpm':float, 'cpc':float, 'cpa':float})

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    w

In [14]:
df.head()

,yymm,child_ssp,device,domain,req,res,imp,click,cpc,ctr,gross_sales,cpm,ctv1,ctv2,cvr1,cpa,cvr2_ctv1_2,rank
0,2019-10,25,PC,native.article.jp.msn.com.example.net,824995135,510729928,40785974,85818,98.827589,0.002104,8481186,207.943692,902,755,0.010511,5118.398310,0.019308,1
1,2019-10,25,SP_web,japanese.engadget.com,30760969,26428239,2630497,11641,66.626664,0.004425,775601,294.849604,165,41,0.014174,3765.053398,0.017696,2
2,2019-10,25,SP_web,native.article.jp.msn.com.example.net,60988654,57628037,2017561,26661,64.320918,0.013214,1714860,849.966866,334,122,0.012528,3760.657895,0.017104,3
3,2019-10,25,SP_web,www.sanspo.com,58313904,51408423,1609676,20849,58.802437,0.012952,1225972,761.626563,259,92,0.012423,3492.797721,0.016835,4
4,2019-10,25,SP_web,www.sankei.com,109582749,101877679,1534093,27014,69.346302,0.017609,1873321,1221.126099,401,183,0.014844,3207.741438,0.021618,5


In [15]:
#PC&SP（アプリ面以外）
list = {1:1, 3:2 ,9:3 ,10:4 ,11:5 ,13:6 ,14:7 ,15:8 ,16:9 ,17:10 ,18:11 ,19:12 ,20:13 ,21:14 ,22:15 ,23:16 ,25:17 ,26:18 ,27:19 ,29:20 ,30:21 ,31:22, 32:23, 33:24, 34:25}
for i, sheet in list.items():
    df_new = df[(df['child_ssp'] == i) & ((df['device'] == 'PC') | (df['device'] == 'SP_web'))]
    gs = GSheet()
    gs.add_data(url_web, sheet, df_new, [0, 0], True)

INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/sheets/v4/rest
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/13T1tGrVGauIAo8wvZTkZuYtiEks4yGjUH1Rvqalim-o?alt=json
INFO:googleapiclient.discovery:URL being requested: PUT https://sheets.googleapis.com/v4/spreadsheets/13T1tGrVGauIAo8wvZTkZuYtiEks4yGjUH1Rvqalim-o/values/Google%21A1%3AS1?valueInputOption=USER_ENTERED&alt=json
INFO:googleapiclient.discovery:URL being requested: PUT https://sheets.googleapis.com/v4/spreadsheets/13T1tGrVGauIAo8wvZTkZuYtiEks4yGjUH1Rvqalim-o/values/Google%21A2%3AS4837?valueInputOption=USER_ENTERED&alt=json
INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_

In [16]:
df_new.head()

,yymm,child_ssp,device,domain,req,res,imp,click,cpc,ctr,gross_sales,cpm,ctv1,ctv2,cvr1,cpa,cvr2_ctv1_2,rank
63775,2019-10,34,PC,native.article.jp.msn.com.example.net,89591588,54416575,15324582,42342,20.315880,0.002763,860215,56.133016,378,51,0.008927,2005.163170,0.010132,1
63776,2019-10,34,SP_web,news.livedoor.com,226398382,202630464,10302050,34084,11.668613,0.003308,397713,38.605229,276,146,0.008098,942.447867,0.012381,2
63777,2019-10,34,PC,tabelog.com,24948944,13186098,7373718,4675,97.741604,0.000634,456942,61.969009,5,5,0.001070,45694.200000,0.002139,3
63778,2019-10,34,SP_web,tabelog.com,30212061,24069189,5908631,16801,44.941611,0.002843,755064,127.790008,133,161,0.007916,2568.244898,0.017499,4
63779,2019-10,34,SP_web,the-ans.jp,48388043,41845374,5316267,24828,39.411511,0.004670,978509,184.059416,186,78,0.007492,3706.473485,0.010633,5


In [17]:
#アプリ面
list = {1:1, 3:2 ,9:3 ,10:4 ,11:5 ,13:6 ,14:7 ,15:8 ,16:9 ,17:10 ,18:11 ,19:12 ,20:13 ,21:14 ,22:15 ,23:16 ,25:17 ,26:18 ,27:19 ,29:20 ,30:21 ,31:22, 32:23, 33:24, 34:25}
for i, sheet in list.items():
    df_new = df[(df['child_ssp'] == i) & (df['device'] == 'SP_app')]
    gs = GSheet()
    gs.add_data(url_app, sheet, df_new, [0, 0], True)

INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/sheets/v4/rest
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1Hm7PMqq0npGb0qDXwqCGivoTE7QTThh7ae18QVuspE4?alt=json
INFO:googleapiclient.discovery:URL being requested: PUT https://sheets.googleapis.com/v4/spreadsheets/1Hm7PMqq0npGb0qDXwqCGivoTE7QTThh7ae18QVuspE4/values/Google%21A1%3AS1?valueInputOption=USER_ENTERED&alt=json
INFO:googleapiclient.discovery:URL being requested: PUT https://sheets.googleapis.com/v4/spreadsheets/1Hm7PMqq0npGb0qDXwqCGivoTE7QTThh7ae18QVuspE4/values/Google%21A2%3AS167?valueInputOption=USER_ENTERED&alt=json
INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_t

gm = GMail()


mail_body = (
    f"""

    みなさま、
    
    お疲れ様です、赤田です。

    前月分のSSP別上位ドメインのリストを更新しました。
    ご査証下さい。
    
    PC・SP（アプリ除く）
    https://docs.google.com/spreadsheets/d/1EOVUj4l9XVefOaL-JykI-VJ9LZ9EtS4IVE9jfdx5JnE/edit#gid=526100795
    
    アプリ面
    https://docs.google.com/spreadsheets/d/1MgugxcQc4Cnzi36S-3-GCZ9_UZgo4dTvhjqa3-xI_yw/edit#gid=1192836758
    
    なお、過去分は、下記フォルダに保存しています。
    https://drive.google.com/drive/folders/1A55oM9DuS7F26CId_GbzvbJBEk_tOAJv
    
    
    """
)

atr = gm.send_gmail(subject=f'SSP別上位ドメインのリスト更新', 
                    message_text=mail_body,
                    to_addrs=['tayasuda@so-netmedia.jp','reina_murano@so-netmedia.jp','saya_kouno@so-netmedia.jp'],
                    cc_addrs=['ashimatani@so-netmedia.jp','dm@so-netmedia.jp','yusuke_akada@so-netmedia.jp'])